In [314]:
import sys
!{sys.executable} -m pip install sodapy

import pandas as pd
from sodapy import Socrata
import numpy as np

#retrieving Food Inspections data using Socrata and converting to DF
client = Socrata("data.cityofchicago.org", None)
results = client.get("cwig-ma7x", limit=2000) #limit set for ease of processing use
inspections_df = pd.DataFrame.from_records(results)

inspections_df.head(n=10)

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,7,4454,39,397,18,7146 W DIVERSEY AVE,TAQUERIA LA FIESTA,CHICAGO,TAQUERIA LA FIESTA,Restaurant,...,Canvass,41.9308726678483,1799983,"{'type': 'Point', 'coordinates': [-87.80558156...",-87.80558156527526,Out of Business,Risk 1 (High),IL,NaN,60707
1,46,14926,41,290,37,226 W SCHILLER ST,CATHERINE COOK SCHOOL,CHICAGO,HANDCUT FOODS,Restaurant,...,Canvass,41.90775713038418,2372990,"{'type': 'Point', 'coordinates': [-87.63572939...",-87.63572939840577,Pass w/ Conditions,Risk 1 (High),IL,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60610
2,29,22243,34,653,77,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS,CHICAGO,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),...,License,42.01177844204009,2500760,"{'type': 'Point', 'coordinates': [-87.80856351...",-87.80856351116931,Not Ready,Risk 1 (High),IL,NaN,60631
3,42,22212,33,378,74,2320 W 113TH PL,SMITH VILLAGE,CHICAGO,SMITH VILLAGE,Long Term Care,...,Canvass,41.68722880202984,2319834,"{'type': 'Point', 'coordinates': [-87.67964685...",-87.6796468584057,Pass w/ Conditions,Risk 1 (High),IL,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60643
4,25,21186,38,691,57,1060 W ADDISON ST,CATALINA CLUB,CHICAGO,CATALINA CLUB,Restaurant,...,License,41.94731748901495,2647347,"{'type': 'Point', 'coordinates': [-87.65641794...",-87.65641794764645,Pass,Risk 3 (Low),IL,NaN,60613
5,29,22243,34,653,77,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS,CHICAGO,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),...,Canvass,42.01177844204009,2500760,"{'type': 'Point', 'coordinates': [-87.80856351...",-87.80856351116931,Fail,Risk 1 (High),IL,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60631
6,50,22532,20,656,36,5806-5808 N MILWAUKEE AVE,KAZIK BAKERY,CHICAGO,KOVALA CORP,Bakery,...,Canvass,41.986971746707326,85114,"{'type': 'Point', 'coordinates': [-87.77824042...",-87.77824042057848,Pass w/ Conditions,Risk 2 (Medium),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60646
7,28,21569,14,248,32,3002 S PULASKI RD,LA CASE DEL BORREGO,CHICAGO,"LA CASA DEL BORREGO, INC.",Restaurant,...,License,41.838571723634004,2658561,"{'type': 'Point', 'coordinates': [-87.72454334...",-87.72454334680803,Fail,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60623
8,31,21554,6,573,70,7801 S WOLCOTT AVE,HEAVEN'S LEARNING GARDEN,CHICAGO,HEAVEN'S LEARNING GARDEN,NaN,...,License,41.75199654639111,2247035,"{'type': 'Point', 'coordinates': [-87.67060879...",-87.67060879151809,Pass,Risk 1 (High),IL,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,60620
9,7,4454,39,397,18,7146 W DIVERSEY AVE,EL TACONAZO LA FIESTA,CHICAGO,EL TACONAZO LA FIESTA,Restaurant,...,Complaint Re-Inspection,41.9308726678483,1799983,"{'type': 'Point', 'coordinates': [-87.80558156...",-87.80558156527526,Fail,Risk 1 (High),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60707


In [297]:
#Iterate through to read the full dataset
initial = 2000
while results:
    results = client.get("cwig-ma7x", limit=2000, offset=initial)
    inspections_df = inspections_df.append(pd.DataFrame.from_records(results))
    initial += 2000

inspections_df.count()

:@computed_region_43wa_7qmu    222888
:@computed_region_6mkv_f3dw    226168
:@computed_region_awaf_s7ux    222888
:@computed_region_bdys_3d7i    226137
:@computed_region_vrxf_vc4k    222888
address                        226999
aka_name                       224370
city                           226796
dba_name                       226999
facility_type                  221988
inspection_date                226999
inspection_id                  226999
inspection_type                226998
latitude                       226168
license_                       226981
location                       226168
longitude                      226168
results                        226999
risk                           226909
state                          226937
violations                     167102
zip                            226915
dtype: int64

In [315]:
#drop uneccesary computed columns - consider dropping by column index; perform before outputing to csv
inspections_df = inspections_df.drop(columns=[':@computed_region_43wa_7qmu', ':@computed_region_6mkv_f3dw', ':@computed_region_awaf_s7ux'
                            ,':@computed_region_bdys_3d7i', ':@computed_region_vrxf_vc4k', 'location', 'city', 'state', 'aka_name'])
inspections_df.head()

,address,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,longitude,results,risk,violations,zip
0,7146 W DIVERSEY AVE,TAQUERIA LA FIESTA,Restaurant,2019-04-05T00:00:00.000,2282654,Canvass,41.9308726678483,1799983,-87.80558156527526,Out of Business,Risk 1 (High),NaN,60707
1,226 W SCHILLER ST,HANDCUT FOODS,Restaurant,2019-04-05T00:00:00.000,2282677,Canvass,41.90775713038418,2372990,-87.63572939840577,Pass w/ Conditions,Risk 1 (High),10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60610
2,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2019-04-05T00:00:00.000,2282667,License,42.01177844204009,2500760,-87.80856351116931,Not Ready,Risk 1 (High),NaN,60631
3,2320 W 113TH PL,SMITH VILLAGE,Long Term Care,2019-04-05T00:00:00.000,2282669,Canvass,41.68722880202984,2319834,-87.6796468584057,Pass w/ Conditions,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60643
4,1060 W ADDISON ST,CATALINA CLUB,Restaurant,2019-04-05T00:00:00.000,2282673,License,41.94731748901495,2647347,-87.65641794764645,Pass,Risk 3 (Low),NaN,60613


In [316]:
#Drop rows with null data
inspections_df.dropna()
inspections_df.count()

address            2000
dba_name           2000
facility_type      1976
inspection_date    2000
inspection_id      2000
inspection_type    2000
latitude           1988
license_           2000
longitude          1988
results            2000
risk               1998
violations         1530
zip                1996
dtype: int64

In [317]:
#Checking whether there are any duplicates
#If duplicates exist, then use .drop_duplicates()
d = inspections_df.duplicated()

duplicates_amount = 0 
duplicates = []
try:
    for i in range(1,len(d)):
        if d[i] == True:
            duplicates_amount = duplicates_amount + 1
            duplicates.append(i)
            print("Duplicates Amount: ",duplicates_amount) 
except:
    print("No duplicates in this dataset")

In [318]:
#Remove trailing characters from date field
inspections_df["inspection_date"] = inspections_df.inspection_date.str.split('T').str[0]
inspections_df.inspection_date.head(n=10)

0    2019-04-05
1    2019-04-05
2    2019-04-05
3    2019-04-05
4    2019-04-05
5    2019-04-05
6    2019-04-05
7    2019-04-05
8    2019-04-05
9    2019-04-05
Name: inspection_date, dtype: object

In [319]:
#Exclude unsuccessful inspections
inspections_df = inspections_df[~inspections_df.results.isin(["Business Not Located", "Out of Business"])] 
inspections_df.count()

address            1898
dba_name           1898
facility_type      1882
inspection_date    1898
inspection_id      1898
inspection_type    1898
latitude           1887
license_           1898
longitude          1887
results            1898
risk               1896
violations         1529
zip                1894
dtype: int64

In [320]:
#Format data types - can use as_type() to convert 
inspections_df.dtypes

address            object
dba_name           object
facility_type      object
inspection_date    object
inspection_id      object
inspection_type    object
latitude           object
license_           object
longitude          object
results            object
risk               object
violations         object
zip                object
dtype: object

In [321]:
#Rename columns due to sodapy conversion
inspections_df = inspections_df.rename(columns={"license_":"License", "address":"Address",
                               "dba_name":"DBA Name", "facility_type":"Facility Type", "inspection_date":"Inspection Date",
                              "inspection_id":"Inspection ID", "inspection_type":"Inspection Type", "latitude":"Latitude",
                              "longitude":"Longitude", "results":"Results", "risk":"Risk", 
                              "violations":"Violations", "zip":"Zip"})
inspections_df.head()

,Address,DBA Name,Facility Type,Inspection Date,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Violations,Zip
1,226 W SCHILLER ST,HANDCUT FOODS,Restaurant,2019-04-05,2282677,Canvass,41.90775713038418,2372990,-87.63572939840577,Pass w/ Conditions,Risk 1 (High),10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60610
2,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2019-04-05,2282667,License,42.01177844204009,2500760,-87.80856351116931,Not Ready,Risk 1 (High),NaN,60631
3,2320 W 113TH PL,SMITH VILLAGE,Long Term Care,2019-04-05,2282669,Canvass,41.68722880202984,2319834,-87.6796468584057,Pass w/ Conditions,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60643
4,1060 W ADDISON ST,CATALINA CLUB,Restaurant,2019-04-05,2282673,License,41.94731748901495,2647347,-87.65641794764645,Pass,Risk 3 (Low),NaN,60613
5,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2019-04-05,2282666,Canvass,42.01177844204009,2500760,-87.80856351116931,Fail,Risk 1 (High),10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60631


In [326]:
#Separate DF into Inspections and Violations
inspections = inspections_df.loc[:, inspections_df.columns != 'Violations']
inspections.head()

,Address,DBA Name,Facility Type,Inspection Date,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Zip
1,226 W SCHILLER ST,HANDCUT FOODS,Restaurant,2019-04-05,2282677,Canvass,41.90775713038418,2372990,-87.63572939840577,Pass w/ Conditions,Risk 1 (High),60610
2,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2019-04-05,2282667,License,42.01177844204009,2500760,-87.80856351116931,Not Ready,Risk 1 (High),60631
3,2320 W 113TH PL,SMITH VILLAGE,Long Term Care,2019-04-05,2282669,Canvass,41.68722880202984,2319834,-87.6796468584057,Pass w/ Conditions,Risk 1 (High),60643
4,1060 W ADDISON ST,CATALINA CLUB,Restaurant,2019-04-05,2282673,License,41.94731748901495,2647347,-87.65641794764645,Pass,Risk 3 (Low),60613
5,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2019-04-05,2282666,Canvass,42.01177844204009,2500760,-87.80856351116931,Fail,Risk 1 (High),60631


In [327]:
violations = inspections_df.loc[:, inspections_df.columns != 'Inspection Date']
violations.head()

,Address,DBA Name,Facility Type,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Violations,Zip
1,226 W SCHILLER ST,HANDCUT FOODS,Restaurant,2282677,Canvass,41.90775713038418,2372990,-87.63572939840577,Pass w/ Conditions,Risk 1 (High),10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60610
2,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2282667,License,42.01177844204009,2500760,-87.80856351116931,Not Ready,Risk 1 (High),NaN,60631
3,2320 W 113TH PL,SMITH VILLAGE,Long Term Care,2282669,Canvass,41.68722880202984,2319834,-87.6796468584057,Pass w/ Conditions,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60643
4,1060 W ADDISON ST,CATALINA CLUB,Restaurant,2282673,License,41.94731748901495,2647347,-87.65641794764645,Pass,Risk 3 (Low),NaN,60613
5,7248-7252 W TOUHY AVE,CHILDREN'S CAMPUS CENTER,Daycare (2 - 6 Years),2282666,Canvass,42.01177844204009,2500760,-87.80856351116931,Fail,Risk 1 (High),10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,60631


In [329]:
#Create two output tables and third table with entire target schema
import os.path
root_path = os.path.dirname(os.getcwd())

#Export to CSV
inspections_df.to_csv(os.path.join(root_path, "Food Inspections Data/Full Food Inspections.csv"), index=False)
inspections.to_csv(os.path.join(root_path, "Food Inspections Data/Inspections.csv"), index=False)
violations.to_csv(os.path.join(root_path, "Food Inspections Data/Violations.csv"), index=False)

In [404]:
#Unit Testing ETL script
import unittest

class TestETL(unittest.TestCase):
    def test_empty_dataframe(self):
        """
        Tests if dataframe is empty
        """
        self.assertIsNotNone(inspections_df)
        
    def test_output_tables(self):
        """
        Tests that the Violations field is not in the inspections table and
        that the Inspection ID field is not in the violations table
        """
        self.assertTrue(inspections.columns.any != 'Violations')
        self.assertTrue(violations.columns.any != "Inspection ID")
        
    def test_filtered_fields(self):
        """
        Tests that unwanted fields get filtered out of DF
        """
        self.assertNotIn("Business Not Located", inspections_df.Results.values)
        self.assertNotIn("Out of Business", inspections_df.Results.values)
        self.assertNotIn(('location', 'city', 'state', 'aka_name'), inspections_df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)

test_empty_dataframe (__main__.TestETL) ... ok
test_filtered_fields (__main__.TestETL) ... ok
test_output_tables (__main__.TestETL) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.014s

OK
